<a href="https://colab.research.google.com/github/olexandr7/numismatic_data_tallinn_city_museum/blob/main/numismatic_data_muis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import xml.etree.ElementTree as ET
import pandas as pd

# Path to your XML file
xml_file = "lido2.xml"
ns = {"lido": "http://www.lido-schema.org"}

# Parse the XML
tree = ET.parse(xml_file)
root = tree.getroot()

In [53]:
rows = []
for record in root.findall("lido:lido", ns):
    recid = record.findtext(".//lido:lidoRecID[@lido:type='local']", namespaces=ns)
    muuseumikogu = record.findtext(".//lido:classification[@lido:type='muuseumikogu']/lido:term", namespaces=ns)
    number = record.findtext(".//lido:workID[@lido:type='museaali täisnumber']", namespaces=ns)
    nimetus = record.findtext(".//lido:titleSet[@lido:type='museaali nimetus']/lido:appellationValue", namespaces=ns)
    olemus = record.findtext(".//lido:objectWorkType[@lido:type='olemus']/lido:term", namespaces=ns)
    seisund = record.findtext(".//lido:displayStateEditionWrap/lido:displayState", namespaces=ns)
# --- Dateering ---
    date_start = record.findtext(".//lido:eventDate/lido:date/lido:earliestDate", namespaces=ns)
    date_end = record.findtext(".//lido:eventDate/lido:date/lido:latestDate", namespaces=ns)
    if date_start and date_end:
        dateering = date_start if date_start == date_end else f"{date_start}–{date_end}"
    else:
        dateering = date_start or date_end

# --- Originaal + Originaal_tüüp ---
    originaal = record.findtext(".//lido:eventDescriptionSet/lido:descriptiveNoteValue", namespaces=ns)
    if originaal:
        originaal_lower = originaal.lower()
        originaal_tüüp = "koopia" if "koopia" in originaal_lower else "originaal"
    else:
        originaal_tüüp = None

# --- Kommentaar (from objectDescriptionSet) ---
    kommentaar = record.findtext(".//lido:objectDescriptionSet/lido:descriptiveNoteValue", namespaces=ns)

# --- Tehnika (skip first two terms) ---
    tehnika_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='tehnika']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if len(terms) > 2:
            tehnika_terms.extend(terms[2:])
        elif len(terms) > 1:
            tehnika_terms.extend(terms[1:])
    tehnika = "; ".join(dict.fromkeys(tehnika_terms)) if tehnika_terms else None

# --- Materjal (only most specific term) ---
    materjal_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='materjal']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if terms:
            materjal_terms.append(terms[-1])
    materjal = "; ".join(dict.fromkeys(materjal_terms)) if materjal_terms else None

# --- Mõõdud ---
    mõõdud_type = record.findtext(".//lido:measurementType", namespaces=ns)
    mõõdud_val = record.findtext(".//lido:measurementValue", namespaces=ns)
    mõõdud_unit = record.findtext(".//lido:measurementUnit", namespaces=ns)
    mõõdud = f"{mõõdud_type} {mõõdud_val} {mõõdud_unit}" if mõõdud_type and mõõdud_val and mõõdud_unit else mõõdud_val

# --- Riik (first non-empty under politicalEntity="riik") ---
    riik = None
    for el in record.findall(".//lido:eventPlace/lido:place[@lido:politicalEntity='riik']/lido:namePlaceSet/lido:appellationValue", ns):
        if el is not None and el.text:
            cand = el.text.strip()
            if cand and cand != "[]":
                riik = cand
                break

# --- Link ---
    link = f"https://www.muis.ee/museaalview/{recid}" if recid else None

    rows.append({
        "RecID": recid,
        "Link": link,
        "Muuseumikogu": muuseumikogu,
        "Number": number,
        "Nimetus": nimetus,
        "Olemus": olemus,
        "Dateering": dateering,
        "Originaal": originaal,
        "Originaal_tüüp": originaal_tüüp,
        "Kommentaar": kommentaar,
        "Seisund": seisund,
        "Tehnika": tehnika,
        "Materjal": materjal,
        "Mõõdud": mõõdud,
        "Riik": riik
    })

# Create DataFrame
df = pd.DataFrame(rows)


In [ ]:
# Save automatically to CSV
output_file = "muis_records.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

In [55]:
print(df.iloc[0])

RecID                                             1202822
Link              https://www.muis.ee/museaalview/1202822
Muuseumikogu                                 numismaatika
Number                                      TLM _ 27664 N
Nimetus                                             Medal
Olemus                                              medal
Dateering                                       1990–2000
Originaal                                            None
Originaal_tüüp                                       None
Kommentaar                                           None
Seisund                                               hea
Tehnika                                         vermimine
Materjal                                     valge metall
Mõõdud                                    läbimõõt 5.0 cm
Riik                                              Euroopa
Name: 0, dtype: object


In [56]:
print(df.to_string(index=False))


  RecID                                    Link Muuseumikogu             Number                                               Nimetus Olemus Dateering                                               Originaal Originaal_tüüp                                Kommentaar  Seisund   Tehnika       Materjal           Mõõdud                            Riik
1202822 https://www.muis.ee/museaalview/1202822 numismaatika      TLM _ 27664 N                                                 Medal  medal 1990–2000                                                    None           None                                      None      hea vermimine   valge metall  läbimõõt 5.0 cm                         Euroopa
1202821 https://www.muis.ee/museaalview/1202821 numismaatika      TLM _ 27673 N                                               kopikas   münt      1771                                                    None           None                                      None rahuldav vermimine           vask  läbimõõ